## Plot GPS Coordinates to Map 
- varify coastal ports
- https://towardsdatascience.com/easy-steps-to-plot-geographic-data-on-a-map-python-11217859a2db
- https://www.openstreetmap.org/export#map=2/23.9/26.0

In [43]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [72]:
#load datasets
coastal = pd.read_csv('CoastalPorts.csv')
other = pd.read_csv('OtherPorts.csv')

In [73]:
coastal.head()

,Country Code,Port Location,Port Name,Port Authority,Address,Phone,Fax,800 Number,Email,Web Site,Latitude,Longitude,UN/LOCODE,Port Type,Port Size,Country
0,ALB,Durres,Port of Durres,Durres Port Authority,Kapitenerija e Portit\nL Nr 1 Rruga Tregtare\n...,355 52 23115,355 52 22028,NaN,apd@san.com.al,NaN,"41° 18' 28"" N","19° 27' 17"" E",ALDRZ,Seaport,Small,NaN
1,ALB,Sarande,Port of Sarande,Sarande Port Authority,Port Office\nSarande Port\nSarande\nAlbania,355 85 25827,355 85 25827,NaN,NaN,NaN,"39° 52' 19"" N","20° 0' 20"" E",ALSAR,"Pier, Jetty or Wharf",Small,NaN
2,ALB,Vlore,Port of Vlore,Vlore Drejtoria e Portit Detar,Albania,NaN,NaN,NaN,porti-vlore@aul.com.al,NaN,"40° 28' 7"" N","19° 27' 36"" E",ALVOA,Seaport,Small,NaN
3,DZA,Algiers,Port of Algiers,Entreprise Portuaire d'Alger,"02 Rue d'Angkor\nBP 259\nAlgiers, Gare 259\nAl...",213 21 423614,213 21 423603,NaN,epal@portalger.com.dz,www.portalger.com.dz,"36° 46' 25"" N","3° 4' 2"" E",DZALG,Deepwater Seaport,Large,NaN
4,DZA,Annaba,Port of Annaba,Entreprise Portuaire de Annaba,Mole Cigogne\nQuai Nord\nAnnaba 1232 - 23000\n...,213 38 86 3143,213 38 86 54 15,NaN,epan@annaba-port.com,www.annaba-port.com,"36° 54' 2"" N","7° 46' 21"" E",DZAAE,Seaport,Medium,NaN


In [79]:
#subset data into country code, port name, lat, long, etc
coastal_subset = coastal.drop(['Port Authority', 'Address', 'Phone', 'Fax', '800 Number', 'Email', 'Web Site', 'Country'], axis=1)
coastal_subset.head()

,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size
0,ALB,Durres,Port of Durres,"41° 18' 28"" N","19° 27' 17"" E",ALDRZ,Seaport,Small
1,ALB,Sarande,Port of Sarande,"39° 52' 19"" N","20° 0' 20"" E",ALSAR,"Pier, Jetty or Wharf",Small
2,ALB,Vlore,Port of Vlore,"40° 28' 7"" N","19° 27' 36"" E",ALVOA,Seaport,Small
3,DZA,Algiers,Port of Algiers,"36° 46' 25"" N","3° 4' 2"" E",DZALG,Deepwater Seaport,Large
4,DZA,Annaba,Port of Annaba,"36° 54' 2"" N","7° 46' 21"" E",DZAAE,Seaport,Medium


In [59]:
## Test
#change coordinates to decimals
lat = '''7° 5' 16" S'''
deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)

-7.087778

In [58]:
## Test
#change coordinates to decimals
lon = '''97° 50' 2" W'''
deg, minutes, seconds, direction =  re.split('[°\'"]', lon)
round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)

-97.833889

In [81]:
#convert degree minute second to degree decimal for latitudes
lat_list = []
for latitude in coastal_subset['Latitude']:
    #print(latitude)
    lat = latitude
    deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
    lat = round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)
    #print(lat)
    lat_list.append(lat)
coastal_subset['Latitude'] = lat_list

In [84]:
#convert degree minute second to degree decimal for latitudes
lon_list = []
for longitude in coastal_subset['Longitude']:
    #print(longitude)
    lon = longitude
    deg, minutes, seconds, direction =  re.split('[°\'"]', lon)
    lon = round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)
    #print(lon)
    lon_list.append(lon)
coastal_subset['Longitude'] = lon_list

In [85]:
coastal_subset.head()

,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size
0,ALB,Durres,Port of Durres,41.307778,19.454722,ALDRZ,Seaport,Small
1,ALB,Sarande,Port of Sarande,39.871944,20.005556,ALSAR,"Pier, Jetty or Wharf",Small
2,ALB,Vlore,Port of Vlore,40.468611,19.460000,ALVOA,Seaport,Small
3,DZA,Algiers,Port of Algiers,36.773611,3.067222,DZALG,Deepwater Seaport,Large
4,DZA,Annaba,Port of Annaba,36.900556,7.772500,DZAAE,Seaport,Medium


In [86]:
#define bounding box - area defined by two longitudes and two latitudes
#BBox = ((coastal_subset.Longitude.min(), 
#         coastal_subset.Longitude.max(), 
#         coastal_subset.Latitude.min(), 
#         coastal_subset.Latitude.max()))
#BBox

#load map image 
#worldmap = plt.imread('worldmap.png')

#plot coordinates as scatter points on 'worldmap'
#set up the X-axis and Y-axis per bounding box ‘BBox’
#fig, ax = plt.subplots(figsize = (15,12))
#ax.scatter(coastal_subset.Longitude, coastal_subset.Latitude, zorder=1, alpha= 0.2, c='b', s=10)
#ax.set_title('Plotting Coastal Port Data on World Map')
#ax.set_xlim(BBox[0],BBox[1])
#ax.set_ylim(BBox[2],BBox[3])
#ax.imshow(worldmap, zorder=0, extent = BBox, aspect= 'equal')

(-179.4875, 179.363333, -54.932778, 78.927778)

In [95]:
# Import necessary packages
import os 
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

In [98]:
m = folium.Map(location=[41.307778, 19.454722])
m